** Cleaning Stock Market dataset

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

- load the dataset

In [48]:
Original_FILE_PATH = Path("../data/raw/orignial_sp500.csv")
df= pd.read_csv(Original_FILE_PATH,skiprows=2)


- Rename the columns

In [49]:
columns_names= ["Date","Open","High","Low","Close","Volume"]

if list(df.columns) != columns_names:
    print("Renaming done")
    df.columns=columns_names
else:
    print("Names are correct")    

Renaming done


- Conver Date column's data type

In [50]:
if df["Date"].dtype != np.dtype('datetime64[ns]'):
    print("Converting datatype...")
    df["Date"]= pd.to_datetime(df["Date"])
else:
    print("Correct data type")

Converting datatype...


Ensure all other columns are numeric 

In [51]:
numeric_columns = ["Open","High","Low","Close","Volume"]

for col in numeric_columns:
    if not pd.api.types.is_numeric_dtype(df[col]):
        print(f"Converted {col} to Numeric..")
        df[col]=pd.to_numeric(df[col])
    else:
        print(f"{col} has correct data type")
print("\n ......  Perfect!  ....")

Open has correct data type
High has correct data type
Low has correct data type
Close has correct data type
Volume has correct data type

 ......  Perfect!  ....


- Sort & Set index

In [52]:
# sort
if not df["Date"].is_monotonic_increasing:
    print("Sort..")
    pd.sort_values(df["Date"])
else:
    print("Already sorted")
# set index
if df.index.name != "Date":
    print("\n Index set")
    df=df.set_index(df["Date"])
else:
    print("Already set")

Already sorted

 Index set


- Save version 2

In [53]:
PROCESSED_DIRE = Path("../data/processed")
PROCESSED_DIRE.mkdir(exist_ok=True)

version_2_dire = PROCESSED_DIRE/"version_2_sp500.csv"

if not version_2_dire.exists():
    print("Verion 2 added")
    df.to_csv(version_2_dire)
else:
    print("Version 2 already exists")


Version 2 already exists


- Check the existent of misiing data

In [54]:
missing_data = df.isna().sum()

if missing_data.sum() == 0:
    print("No missing data foumd")
else:
    print("Data should be cleaned")

No missing data foumd


- Check the existent of duplicated values

In [55]:
duplicated = df.duplicated().sum()

if duplicated.sum() == 0:
    print("No duplication")
else:
    print("Duplication found and should be handled")

No duplication


- Index column : duplicate values are prohibted
    * Check if there are duplicate values in Date column

In [56]:
index_dup_checker = df["Date"].duplicated().sum()

if index_dup_checker.sum() == 0:
    print("PERFECT!, no duplication in index column")
else:
    print("Duplication in index column found and should be handled")

PERFECT!, no duplication in index column


- Check valid prices
    * low < open
    * high > close   etc...

In [57]:
invalid_prices = df[  (df["Open"]<df["Low"]) | (df["High"]<df["Close"]) | (df["High"]<df["Low"])  | (df["High"]<df["Open"]) | (df["Low"] > df["Close"])     ]

if invalid_prices.empty:
    print("Prices are Valid")
else: 
    print("Prices are Invalid")

Prices are Valid


- Check valid Volume
   * Not negative

In [58]:

invalid_volume= df [ (df["Volume"] < 0 )]

if invalid_volume.empty:
    print("Volumes are Valid")
else: 
    print("Volumes are Invalid")

Volumes are Valid


- Check Impossible prices
  * Not Negative
  * Not Zero

In [59]:
columns = ["Open","High","Low","Close"]
impos_prices = df[(df[columns] <= 0).any(axis=1)]
if impos_prices.empty:
    print(" Columns Valid prices")
else: 
    print("Column contain Invalid prices")


 Columns Valid prices


- Save Version 3

In [60]:
version_3_dire = PROCESSED_DIRE/"version_3_sp500.csv"

if not version_3_dire.exists():
    print("Verion 3 added")
    df.to_csv(version_3_dire)
else:
    print("Version 3 already exists")

Version 3 already exists


- Check missing trading dates

In [61]:
trading_dates=pd.date_range(start=df.index.min(), end=df.index.max(),freq="B")
missing_dates= trading_dates.difference(df.index)

if len(missing_data)==0:
    print("No Gaps between trading dates")
else:
    print("There are missing dates")
    display(missing_dates[:20])

There are missing dates


DatetimeIndex(['1950-02-13', '1950-02-22', '1950-04-07', '1950-05-30',
               '1950-07-04', '1950-09-04', '1950-10-12', '1950-11-07',
               '1950-11-23', '1950-12-25', '1951-01-01', '1951-02-12',
               '1951-02-22', '1951-03-23', '1951-05-30', '1951-07-04',
               '1951-09-03', '1951-10-12', '1951-11-06', '1951-11-12'],
              dtype='datetime64[ns]', freq=None)

- Check extreme daily prices
    * SP500 daily price doesn't go beyond 10%

In [62]:
daily_price = df["Close"].pct_change()

extreme = df[daily_price.abs() > 0.10]

if extreme.empty:
    print("Not much difference")
else:
    print("There are huge gabs in some days")
    display(extreme)

There are huge gabs in some days


,Date,Open,High,Low,Close,Volume
Date,,,,,,
1987-10-20,1987-10-20,236.830002,245.619995,216.460007,225.059998,608100000
2008-10-14,2008-10-14,998.010010,1044.310059,972.070007,1009.969971,8161990000
2008-10-29,2008-10-29,930.090027,969.969971,922.260010,939.510010,7077800000


- Check abnormal high/low

In [63]:
range = (df["High"]-df["Low"])/df["Close"]
abnormal = df[range>0.15]

if abnormal.empty:
    print("Normal")
else:
    print("Abnormal")
    display(abnormal)

Abnormal


,Date,Open,High,Low,Close,Volume
Date,,,,,,
1987-10-19,1987-10-19,224.839996,282.700012,224.830002,282.700012,604300000


In [64]:
df["volume30"]= df["Volume"].rolling(30).mean()
df["v_ratio"]= df["Volume"]/df["volume30"]
outlier = df[(df["v_ratio"]>5) | (df["v_ratio"]<0.2)]

if outlier.empty:
    print("No volume outliers")
else:
    print("Outliers exist")

Outliers exist


In [65]:
df.head()

,Date,Open,High,Low,Close,Volume,volume30,v_ratio
Date,,,,,,,,
1950-01-03,1950-01-03,16.66,16.66,16.66,16.66,1260000,NaN,NaN
1950-01-04,1950-01-04,16.85,16.85,16.85,16.85,1890000,NaN,NaN
1950-01-05,1950-01-05,16.93,16.93,16.93,16.93,2550000,NaN,NaN
1950-01-06,1950-01-06,16.98,16.98,16.98,16.98,2010000,NaN,NaN
1950-01-09,1950-01-09,17.08,17.08,17.08,17.08,2520000,NaN,NaN


In [66]:
df = df.drop(columns=["volume30", "v_ratio"], errors="ignore")

- Save version 4

In [67]:
version_4_dire = PROCESSED_DIRE/"version_4_sp500.csv"

if not version_4_dire.exists():
    print("Version 4 added")
    df.to_csv(version_4_dire)
else:
    print("Version 4 already exists")

Version 4 already exists


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 19141 entries, 1950-01-03 to 2026-01-30
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    19141 non-null  datetime64[ns]
 1   Open    19141 non-null  float64       
 2   High    19141 non-null  float64       
 3   Low     19141 non-null  float64       
 4   Close   19141 non-null  float64       
 5   Volume  19141 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 1.5 MB
